import library and set path

In [1]:
import numpy as np
import os
import nrrd
import glob
import math
#import vtk
vtkpaths= glob.glob(os.path.expanduser("~")+'/src/needlevtk/*.save')
datapath = os.path.expanduser("~")+'/DATA/Case  '
print(vtkpaths)



['/home/ubuntu/src/needlevtk/case054vtk.save', '/home/ubuntu/src/needlevtk/case023vtk.save', '/home/ubuntu/src/needlevtk/case060vtk.save', '/home/ubuntu/src/needlevtk/case034vtk.save', '/home/ubuntu/src/needlevtk/case029vtk.save', '/home/ubuntu/src/needlevtk/case026vtk.save', '/home/ubuntu/src/needlevtk/case056vtk.save', '/home/ubuntu/src/needlevtk/case042vtk.save', '/home/ubuntu/src/needlevtk/case012vtk.save', '/home/ubuntu/src/needlevtk/case071vtk.save', '/home/ubuntu/src/needlevtk/case064vtk.save', '/home/ubuntu/src/needlevtk/case035vtk.save', '/home/ubuntu/src/needlevtk/case065vtk.save', '/home/ubuntu/src/needlevtk/case078vtk.save', '/home/ubuntu/src/needlevtk/case022vtk.save', '/home/ubuntu/src/needlevtk/case040vtk.save', '/home/ubuntu/src/needlevtk/case030vtk.save', '/home/ubuntu/src/needlevtk/case027vtk.save', '/home/ubuntu/src/needlevtk/case028vtk.save', '/home/ubuntu/src/needlevtk/case063vtk.save', '/home/ubuntu/src/needlevtk/case017vtk.save', '/home/ubuntu/src/needlevtk/case0

In [10]:
#print full result for debug
def print_full(x):
    np.set_printoptions(threshold=np.inf)
    print(x)
    np.set_printoptions(threshold=1000)
#str2float    
def str2float(list):
    newlist=[]
    n=list.ndim
    if n>1:       
        for j in range(len(list)):
            temp=[float(i) for i in list[j]]
            newlist.append(temp)
    else:
        newlist=[float(i) for i in list]
    return newlist    
#get coordinate transformation matrix
def gettransmat(spacing,origin):
    spacing.append([0.,0.,0.])
    origin.append(1.)
    spacing=np.array(spacing)
    origin=np.array(origin)
    transmat=np.linalg.inv(np.c_[ spacing,origin ])
    return transmat
# transform the coordinate
def transcoord(data,transmat):
    data=data.transpose()
    data=np.append(data,np.array([[1. for i in range(5000)]]),axis=0)
    
    newdata=np.dot(transmat,data)
    newdata=np.rint(newdata)
    newdata=newdata.astype(int)
    
    return newdata       
#get every point intensity from .nrrd file
def getintensity(imgneedles):
    meangray=[]
    intensity=[]
    rng=[-1,0,1]
    for i in range(len(imgneedles)):
        mean=0
        graylevel=[]
        for j in range(5000):
            temp=2000
            #search a patch around the point for the min value
            for x1 in rng:
                for x2 in rng:
                    if nrrddata[0][imgneedles[i][0][j]+x1][imgneedles[i][1][j]+x2][imgneedles[i][2][j]]<temp:
                        temp=nrrddata[0][imgneedles[i][0][j]+x1][imgneedles[i][1][j]+x2][imgneedles[i][2][j]]
            gray=temp
            graylevel.append(gray)
            mean+=gray
        mean/=5000
        meangray.append(mean)
        intensity.append(graylevel)
    return np.array(meangray),np.array(intensity)
#calculate the histogram
def gethistogram(intensity):
    totalhis=[]
    for i in range(len(intensity)):
        upbound=10
        histogram=[]
        while upbound<155:
            num=sum(j < upbound and j>=(upbound-10) for j in intensity[i])/5000
            histogram.append(num)
            upbound+=10
        num=1-sum(histogram)
        histogram.append(num)
        totalhis.append(histogram)
        totalhis2=np.around(totalhis,2)
    return totalhis2         
#calculate mean value of different part of needle
def getsegmean(intensity):
    totalsegmean=[]
    for i in range(len(intensity)):
        upbound=500
        segmean=[]
        while upbound<=5000:
            num=np.mean(intensity[i][upbound-500:upbound])
            segmean.append(num)
            upbound+=500
        totalsegmean.append(segmean)
        totalsegmean2=np.around(totalsegmean,2)
    return totalsegmean2        
# sort the points along Z-axis
def sortseq(needles):
    newseq=[]
    for i in range(len(needles)):
        temp=needles[i][needles[i][:,2].argsort()]
        #print(temp)
        newseq.append(temp)
    return newseq    

totalmean=[]
totalhisto=[]
totalintensity=[]
count=0

for vtkpath in vtkpaths:
    #load needle points
    needles = np.load(vtkpath)
    print(vtkpath)
    #the x-y plane in .vtk and .nrrd file are reverse
    needles[:,:,:2]=needles[:,:,:2]*(-1.)
    needles=sortseq(needles)
    needles=np.array(needles)
    print(needles.shape) 
    #these cases have problems
    if int(vtkpath[-11:-8])==21 or int(vtkpath[-11:-8])==76 or int(vtkpath[-11:-8])==54:
        continue
    #load .nrrd file    
    dirpath=datapath+vtkpath[-11:-8]
    nrrdfiles=glob.glob(dirpath+'/NRRD/Manual_CY/*.nrrd')
    if not nrrdfiles:
        continue
    for nrrdfile in nrrdfiles:
        if not 'label' in nrrdfile:
            nrrdpath=nrrdfile
            break 
    print(nrrdpath)
    count+=1
    nrrddata = nrrd.read(nrrdpath)
    print(nrrddata[1])
    print(nrrddata[1]['space origin'])  
#     nrrdspacing=str2float(np.array(nrrddata[1]['space directions']))
#     nrrdsporig=str2float(np.array(nrrddata[1]['space origin'])) 
#     nrrdsize=str2float(np.array(nrrddata[1]['sizes'])) 

    transmat=gettransmat(nrrdspacing,nrrdsporig)

    imgneedles=[]
    for i in range(len(needles)):
        temp=transcoord(needles[i],transmat)
        imgneedles.append(temp)
    imgneedles=np.array(imgneedles)

    mean,intensity=getintensity(imgneedles)
    histo=gethistogram(intensity)
    segmean=getsegmean(intensity)
    
    if count==1:
        totalmean=mean
        totalhisto=histo
        totalintensity=intensity
        totalsegmean=segmean
    else:
        totalmean=np.concatenate((totalmean, mean), axis=0)
        totalhisto=np.concatenate((totalhisto, histo), axis=0)
        totalintensity=np.concatenate((totalintensity, intensity), axis=0)
        totalsegmean=np.concatenate((totalsegmean, segmean), axis=0)

    print(intensity.shape)

print("ok")


/home/ubuntu/src/needlevtk/case054vtk.save
(24, 5000, 3)
/home/ubuntu/src/needlevtk/case023vtk.save
(14, 5000, 3)
/home/ubuntu/DATA/Case  023/NRRD/Manual_CY/28 AX t2_tse_320_p2.MR._.28.1.2013.02.15.14.06.07.313.43885078.nrrd
{'kinds': ['domain', 'domain', 'domain'], 'type': 'unsigned short', 'dimension': 3, 'encoding': 'gzip', 'keyvaluepairs': {}, 'space': 'left-posterior-superior', 'space directions': [['0.625', '-1.2818962499999991e-010', '0'], ['1.2818962499999991e-010', '0.625', '0'], ['0', '0', '3']], 'space origin': ['-89.950584411621136', '-103.74521636962886', '-160.20606994628903'], 'sizes': [320, 320, 80], 'endian': 'little'}
['-89.950584411621136', '-103.74521636962886', '-160.20606994628903']
(14, 5000)
[[ 0.15  0.11  0.21  0.2   0.06  0.11  0.07  0.08  0.01  0.    0.    0.01
   0.    0.    0.    0.  ]
 [ 0.56  0.06  0.08  0.06  0.07  0.04  0.05  0.02  0.01  0.01  0.    0.
   0.01  0.    0.    0.01]
 [ 0.34  0.17  0.11  0.12  0.09  0.04  0.03  0.04  0.02  0.01  0.01  0.01
 

calculate min max median value and variance

In [11]:
#     index1=[] 
#     for i,x in enumerate(total):
#         if (np.amax(x[0])>nrrdsize[0]-2 or np.amax(x[1])>nrrdsize[1]-2 or np.amax(x[2])>nrrdsize[2]-2 or np.amin(x[1])<0 or np.amin(x[2])<0 or np.amin(x[0])<0):
#             index1.append(i)

totalmin=np.amin(totalintensity,axis=1)
totalmax=np.amax(totalintensity,axis=1)
totalvar=np.var(totalintensity,axis=1)
totalmed=np.median(totalintensity,axis=1)

print(totalmin.shape)
print(totalmax.shape)
print(totalvar.shape)
print(totalmed.shape)  


(875,)
(875,)
(875,)
(875,)


In [12]:
f = open('postotalhisto.save', 'wb')
np.save(f, totalhisto)
f.close
f = open('postotalmean.save', 'wb')
np.save(f, totalmean)
f.close
f = open('postotalintensity.save', 'wb')
np.save(f, totalintensity)
f.close
f = open('postotalsegmean.save', 'wb')
np.save(f, totalsegmean)
f.close
f = open('postotalmin.save', 'wb')
np.save(f, totalmin)
f.close
f = open('postotalvar.save', 'wb')
np.save(f, totalvar)
f.close
f = open('postotalmed.save', 'wb')
np.save(f, totalmed)
f.close
f = open('postotalmax.save', 'wb')
np.save(f, totalmax)
f.close
print('ok')


ok
